In [1]:
#Function to generate a 3-panel plot for input arrays
def plot_array(dem, clim=None, titles=None, cmap='inferno', label=None, overlay=None, fn=None, close_fig=True):
    fig, ax = plt.subplots(1,1, sharex=True, sharey=True, figsize=(10,5))
    alpha = 1.0
    #Gray background
    ax.set_facecolor('0.5')
    #Force aspect ratio to match images
    ax.set(aspect='equal')
    #Turn off axes labels/ticks
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if titles is not None:
        ax.set_title(titles[0])
    #Plot background shaded relief map
    if overlay is not None:
        alpha = 0.7
        ax.imshow(overlay, cmap='gray', clim=(1,255))
    #Plot each array
    im_list = [ax.imshow(dem, clim=clim, cmap=cmap, alpha=alpha)]
    fig.tight_layout()
    fig.colorbar(im_list[0], label=label, extend='both', shrink=0.5)
    if fn is not None:
        fig.savefig(fn, bbox_inches='tight', pad_inches=0, dpi=150)
    if close_fig:
        plt.close(fig)
        
# #RGI uses 50 m bins
# def hist_plot(gf, bin_width=50.0, dz_clim=(-2.0, 2.0), exportcsv=True, csv_ending=''):
#     #print("Generating histograms")
#     #Create bins for full range of input data and specified bin width

#     #NOTE: these counts/areas are for valid pixels only
#     #Not necessarily a true representation of actual glacier hypsometry
#     #Need a void-filled DEM for this

#     z_bin_edges, z_bin_centers = malib.get_bins(gf.z1, bin_width)
#     #Need to compress here, otherwise histogram uses masked values!
#     z1_bin_counts, z1_bin_edges = np.histogram(gf.z1.compressed(), bins=z_bin_edges)
#     z1_bin_areas = z1_bin_counts * gf.res[0] * gf.res[1] / 1E6
#     #RGI standard is integer thousandths of glaciers total area
#     #Should check to make sure sum of bin areas equals total area
#     #z1_bin_areas_perc = 100. * z1_bin_areas / np.sum(z1_bin_areas)
#     z1_bin_areas_perc = 100. * (z1_bin_areas / gf.glac_area_km2)

#     #If we only have one elevation grid with dhdt
#     if gf.z2 is not None:
#         z2_bin_counts, z2_bin_edges = np.histogram(gf.z2.compressed(), bins=z_bin_edges)
#         z2_bin_areas = z2_bin_counts * gf.res[0] * gf.res[1] / 1E6
#         #z2_bin_areas_perc = 100. * z2_bin_areas / np.sum(z2_bin_areas)
#         z2_bin_areas_perc = 100. * (z1_bin_areas / gf.glac_area_km2)
#     else:
#         z2_bin_counts = z1_bin_counts
#         z2_bin_edges = z1_bin_edges
#         z2_bin_areas = z1_bin_areas
#         z2_bin_areas_perc = z1_bin_areas_perc

#     #Create arrays to store output
#     slope_bin_med = np.ma.masked_all_like(z1_bin_areas)
#     slope_bin_mad = np.ma.masked_all_like(z1_bin_areas)
#     aspect_bin_med = np.ma.masked_all_like(z1_bin_areas)
#     aspect_bin_mad = np.ma.masked_all_like(z1_bin_areas)
#     if gf.dhdt is not None:
#         mb_bin_med = np.ma.masked_all_like(z1_bin_areas)
#         np.ma.set_fill_value(mb_bin_med, np.nan)
#         mb_bin_mad = np.ma.masked_all_like(mb_bin_med)
#         mb_bin_mean = np.ma.masked_all_like(mb_bin_med)
#         mb_bin_std = np.ma.masked_all_like(mb_bin_med)
#         dhdt_bin_med = np.ma.masked_all_like(mb_bin_med)
#         dhdt_bin_mad = np.ma.masked_all_like(mb_bin_med)
#         dhdt_bin_mean = np.ma.masked_all_like(mb_bin_med)
#         dhdt_bin_std = np.ma.masked_all_like(mb_bin_med)
#         dhdt_bin_count = np.ma.masked_all_like(mb_bin_med)
#     if gf.vm is not None:
#         vm_bin_med = np.ma.masked_all_like(z1_bin_areas)
#         vm_bin_mad = np.ma.masked_all_like(z1_bin_areas)
#     if gf.H is not None:
#         H_bin_mean = np.ma.masked_all_like(z1_bin_areas)
#         H_bin_std = np.ma.masked_all_like(z1_bin_areas)
#     if gf.emvel is not None:
#         emvel_bin_mean = np.ma.masked_all_like(z1_bin_areas)
#         emvel_bin_std = np.ma.masked_all_like(z1_bin_areas)
#         emvel_bin_med = np.ma.masked_all_like(z1_bin_areas)
#         emvel_bin_mad = np.ma.masked_all_like(z1_bin_areas)
#     if gf.debris_class is not None:
# #         perc_clean = np.ma.masked_all_like(z1_bin_areas)
# #         perc_debris = np.ma.masked_all_like(z1_bin_areas)
# #         perc_pond = np.ma.masked_all_like(z1_bin_areas)
#         debris_thick_med = np.ma.masked_all_like(z1_bin_areas)
#         debris_thick_mad = np.ma.masked_all_like(z1_bin_areas)
# #         dhdt_clean_bin_med = np.ma.masked_all_like(z1_bin_areas)
# #         dhdt_debris_bin_med = np.ma.masked_all_like(z1_bin_areas)
# #         dhdt_pond_bin_med = np.ma.masked_all_like(mz1_bin_areas)

# #         gf.dhdt_clean = np.ma.array(gf.dhdt, mask=~((gf.debris_class == 1).data))
# #         gf.dhdt_debris = np.ma.array(gf.dhdt, mask=~((gf.debris_class == 2).data))
# #         gf.dhdt_pond = np.ma.array(gf.dhdt, mask=~((gf.debris_class == 3).data))

#     if gf.debris_thick_ts is not None:
#         debris_thick_ts_med = np.ma.masked_all_like(z1_bin_areas)
#         debris_thick_ts_mad = np.ma.masked_all_like(z1_bin_areas)
#     if gf.meltfactor_ts is not None:
#         meltfactor_ts_med = np.ma.masked_all_like(z1_bin_areas)
#         meltfactor_ts_mad = np.ma.masked_all_like(z1_bin_areas)

#     #Bin sample count must be greater than this value
#     min_bin_samp_count = 9

#     #Loop through each bin and extract stats
#     idx = np.digitize(gf.z1, z_bin_edges)
#     for bin_n in range(z_bin_centers.size):
#         if gf.dhdt is not None:
#             mb_bin_samp = gf.mb_map[(idx == bin_n+1)]
#             if mb_bin_samp.count() > min_bin_samp_count:
#                 mb_bin_med[bin_n] = malib.fast_median(mb_bin_samp)
#                 mb_bin_mad[bin_n] = malib.mad(mb_bin_samp)
#                 mb_bin_mean[bin_n] = mb_bin_samp.mean()
#                 mb_bin_std[bin_n] = mb_bin_samp.std()
#             dhdt_bin_samp = gf.dhdt[(idx == bin_n+1)]
#             if dhdt_bin_samp.count() > min_bin_samp_count:
#                 dhdt_bin_med[bin_n] = malib.fast_median(dhdt_bin_samp)
#                 dhdt_bin_mad[bin_n] = malib.mad(dhdt_bin_samp)
#                 dhdt_bin_mean[bin_n] = dhdt_bin_samp.mean()
#                 dhdt_bin_std[bin_n] = dhdt_bin_samp.std()
#                 dhdt_bin_count[bin_n] = dhdt_bin_samp.count()
#         if gf.debris_thick is not None:
#             debris_thick_bin_samp = gf.debris_thick[(idx == bin_n+1)]
#             if debris_thick_bin_samp.size > min_bin_samp_count:
#                 debris_thick_med[bin_n] = malib.fast_median(debris_thick_bin_samp)
#                 debris_thick_mad[bin_n] = malib.mad(debris_thick_bin_samp)
        
#         if gf.debris_thick_ts is not None:
#             debris_thick_ts_bin_samp = gf.debris_thick_ts[(idx == bin_n+1)]
#             if debris_thick_ts_bin_samp.size > min_bin_samp_count:
#                 debris_thick_ts_med[bin_n] = malib.fast_median(debris_thick_ts_bin_samp)
#                 debris_thick_ts_mad[bin_n] = malib.mad(debris_thick_ts_bin_samp)
#         if gf.meltfactor_ts is not None:
#             meltfactor_ts_bin_samp = gf.meltfactor_ts[(idx == bin_n+1)]
#             if meltfactor_ts_bin_samp.size > min_bin_samp_count:
#                 meltfactor_ts_med[bin_n] = malib.fast_median(meltfactor_ts_bin_samp)
#                 meltfactor_ts_mad[bin_n] = malib.mad(meltfactor_ts_bin_samp)
                
#         if gf.debris_class is not None:
#             debris_class_bin_samp = gf.debris_class[(idx == bin_n+1)]
#             dhdt_clean_bin_samp = gf.dhdt_clean[(idx == bin_n+1)]
#             dhdt_debris_bin_samp = gf.dhdt_debris[(idx == bin_n+1)]
#             dhdt_pond_bin_samp = gf.dhdt_pond[(idx == bin_n+1)]
#             if debris_class_bin_samp.count() > min_bin_samp_count:
#                 perc_clean[bin_n] = 100. * (debris_class_bin_samp == 1).sum()/debris_class_bin_samp.count()
#                 perc_debris[bin_n] = 100. * (debris_class_bin_samp == 2).sum()/debris_class_bin_samp.count()
#                 perc_pond[bin_n] = 100. * (debris_class_bin_samp == 3).sum()/debris_class_bin_samp.count()
#             if dhdt_clean_bin_samp.count() > min_bin_samp_count:
#                 dhdt_clean_bin_med[bin_n] = malib.fast_median(dhdt_clean_bin_samp)
#             if dhdt_debris_bin_samp.count() > min_bin_samp_count:
#                 dhdt_debris_bin_med[bin_n] = malib.fast_median(dhdt_debris_bin_samp)
#             if dhdt_pond_bin_samp.count() > min_bin_samp_count:
#                 dhdt_pond_bin_med[bin_n] = malib.fast_median(dhdt_pond_bin_samp)
#         if gf.vm is not None:
#             vm_bin_samp = gf.vm[(idx == bin_n+1)]
#             if vm_bin_samp.size > min_bin_samp_count:
#                 vm_bin_med[bin_n] = malib.fast_median(vm_bin_samp)
#                 vm_bin_mad[bin_n] = malib.mad(vm_bin_samp)
#         if gf.H is not None:
#             H_bin_samp = gf.H[(idx == bin_n+1)]
#             if H_bin_samp.size > min_bin_samp_count:
#                 H_bin_mean[bin_n] = H_bin_samp.mean()
#                 H_bin_std[bin_n] = H_bin_samp.std()
#         if gf.emvel is not None:
#             emvel_bin_samp = gf.emvel[(idx == bin_n+1)]
#             if emvel_bin_samp.size > min_bin_samp_count:
#                 emvel_bin_mean[bin_n] = emvel_bin_samp.mean()
#                 emvel_bin_std[bin_n] = emvel_bin_samp.std()
#                 emvel_bin_med[bin_n] = malib.fast_median(emvel_bin_samp)
#                 emvel_bin_mad[bin_n] = malib.mad(emvel_bin_samp)
#         slope_bin_samp = gf.z1_slope[(idx == bin_n+1)]
#         if slope_bin_samp.size > min_bin_samp_count:
#             slope_bin_med[bin_n] = malib.fast_median(slope_bin_samp)
#             slope_bin_mad[bin_n] = malib.mad(slope_bin_samp)
#         aspect_bin_samp = gf.z1_aspect[(idx == bin_n+1)]
#         if aspect_bin_samp.size > min_bin_samp_count:
#             aspect_bin_med[bin_n] = malib.fast_median(aspect_bin_samp)
#             aspect_bin_mad[bin_n] = malib.mad(aspect_bin_samp)

#     if gf.dhdt is not None:
#         dhdt_bin_areas = dhdt_bin_count * gf.res[0] * gf.res[1] / 1E6
#         #dhdt_bin_areas_perc = 100. * dhdt_bin_areas / np.sum(dhdt_bin_areas)
#         dhdt_bin_areas_perc = 100. * (dhdt_bin_areas / gf.glac_area_km2)

#     outbins_header = 'bin_center_elev_m, z1_bin_count_valid, z1_bin_area_valid_km2, z1_bin_area_perc, z2_bin_count_valid, z2_bin_area_valid_km2, z2_bin_area_perc, slope_bin_med, aspect_bin_med'
#     fmt = '%0.1f, %0.0f, %0.3f, %0.2f, %0.0f, %0.3f, %0.2f, %0.2f, %0.2f'
#     outbins = [z_bin_centers, z1_bin_counts, z1_bin_areas, z1_bin_areas_perc, z2_bin_counts, z2_bin_areas, z2_bin_areas_perc, slope_bin_med, aspect_bin_med]
#     if gf.dhdt is not None:
#         outbins_header += ', dhdt_bin_count, dhdt_bin_area_valid_km2, dhdt_bin_area_perc, dhdt_bin_med_ma, dhdt_bin_mad_ma, dhdt_bin_mean_ma, dhdt_bin_std_ma, mb_bin_med_mwea, mb_bin_mad_mwea, mb_bin_mean_mwea, mb_bin_std_mwea'
#         fmt += ', %0.0f, %0.3f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f'
#         outbins.extend([dhdt_bin_count, dhdt_bin_areas, dhdt_bin_areas_perc, dhdt_bin_med, dhdt_bin_mad, dhdt_bin_mean, dhdt_bin_std, \
#                         mb_bin_med, mb_bin_mad, mb_bin_mean, mb_bin_std])
#     if gf.debris_thick is not None:
#         outbins_header += ', debris_thick_med_m, debris_thick_mad_m'
#         fmt += ', %0.2f, %0.2f'
#         debris_thick_med[debris_thick_med == -(np.inf)] = 0.00
#         debris_thick_mad[debris_thick_mad == -(np.inf)] = 0.00
#         outbins.extend([debris_thick_med, debris_thick_mad])
    
#     if gf.debris_thick_ts is not None:
#         outbins_header += ',debris_thick_ts_med_m,debris_thick_ts_mad_m'
#         fmt += ', %0.2f, %0.2f'
#         debris_thick_ts_med[debris_thick_ts_med == -(np.inf)] = 0.00
#         debris_thick_ts_mad[debris_thick_ts_mad == -(np.inf)] = 0.00
#         outbins.extend([debris_thick_ts_med, debris_thick_ts_mad])
#     if gf.meltfactor_ts is not None:
#         outbins_header += ',meltfactor_ts_med_m,meltfactor_ts_mad_m'
#         fmt += ', %0.2f, %0.2f'
#         meltfactor_ts_med[meltfactor_ts_med == -(np.inf)] = 1
#         meltfactor_ts_med[meltfactor_ts_med > 1] = 1
#         meltfactor_ts_med[meltfactor_ts_med <= 0] = 1
#         meltfactor_ts_mad[meltfactor_ts_mad == -(np.inf)] = 0
#         meltfactor_ts_mad[meltfactor_ts_mad > 1] = 0
#         meltfactor_ts_mad[meltfactor_ts_mad <= 0] = 0
#         outbins.extend([meltfactor_ts_med, meltfactor_ts_mad])
    
#     if gf.debris_class is not None:
#         outbins_header += ', perc_debris, perc_pond, perc_clean, dhdt_debris_med, dhdt_pond_med, dhdt_clean_med'
#         fmt += ', %0.2f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f'
#         outbins.extend([perc_debris, perc_pond, perc_clean, dhdt_debris_bin_med, dhdt_pond_bin_med, dhdt_clean_bin_med])
#     if gf.vm is not None:
#         outbins_header += ', vm_med, vm_mad'
#         fmt += ', %0.2f, %0.2f'
#         outbins.extend([vm_bin_med, vm_bin_mad])
#     if gf.H is not None:
#         outbins_header += ', H_mean, H_std'
#         fmt += ', %0.2f, %0.2f'
#         outbins.extend([H_bin_mean, H_bin_std])
# #         outbins_header += ', H_mean, H_std, emvel_mean, emvel_std'
# #         fmt += ', %0.2f, %0.2f, %0.2f, %0.2f'
# #         outbins.extend([H_bin_mean, H_bin_std, emvel_bin_mean, emvel_bin_std])

#     if gf.emvel is not None:
#         outbins_header += ', emvel_mean, emvel_std, emvel_med, emvel_mad'
#         fmt += ', %0.3f, %0.3f, %0.3f, %0.3f'
#         outbins.extend([emvel_bin_mean, emvel_bin_std, emvel_bin_med, emvel_bin_mad])

#     outbins = np.ma.array(outbins).T.astype('float32')
#     np.ma.set_fill_value(outbins, np.nan)
#     outbins = outbins.filled(np.nan)
#     if exportcsv:
#         outbins_fn = os.path.join(outdir_csv, gf.feat_fn[0:8] + csv_ending)
#         np.savetxt(outbins_fn, outbins, fmt=fmt, delimiter=',', header=outbins_header)
    
#     outbins_df = pd.DataFrame(outbins, columns=outbins_header.split(','))
#     return outbins_df, z_bin_edges
# #     return z_bin_edges

In [2]:
#! /usr/bin/env python
"""
Compute debris thickness through sub-debris and temperature inversion methods
"""

import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib
# from imview.lib import pltlib

In [3]:
import debrisglobal.globaldebris_input as debris_prms
from debrisglobal.glacfeat import GlacFeat, create_glacfeat

#INPUT
# topdir='/Users/davidrounce/Documents/Dave_Rounce/HiMAT/DEMs/'
# #Output directory
# outdir = topdir + 'Shean_2019_0213/mb_combined_20190213_nmad_bins/'
# outdir_fig = outdir + '/figures/'
# outdir_csv = outdir + '/csv'

# verbose=False
# extra_layers=True
# min_glac_area_writeout=0
# min_valid_area_perc = 0
# buff_dist = 1000
# bin_width = 5

ts_info_fullfn = debris_prms.ts_fp + debris_prms.roi + '_debris_tsinfo.nc'

print(ts_info_fullfn)

if os.path.exists(debris_prms.ts_fp) == False:
    os.makedirs(debris_prms.ts_fp)

#INPUT
# glac_shp_fn_dict = {'13':input.main_directory + '/../../../HiMAT/RGI/rgi60/13_rgi60_CentralAsia/13_rgi60_CentralAsia.shp',
#                     '14':input.main_directory + '/../../../HiMAT/RGI/rgi60/14_rgi60_SouthAsiaWest/14_rgi60_SouthAsiaWest.shp',
#                     '15':input.main_directory + '/../../../HiMAT/RGI/rgi60/15_rgi60_SouthAsiaEast/15_rgi60_SouthAsiaEast.shp'}
# glac_shp_proj_fp = input.output_fp + 'glac_shp_proj/'
# if os.path.exists(glac_shp_proj_fp) == False:
#     os.makedirs(glac_shp_proj_fp)

# #DEM
# z1_dir_sample = ('/Users/davidrounce/Documents/Dave_Rounce/HiMAT/IceThickness_Farinotti/surface_DEMs_RGI60/' + 
#           'surface_DEMs_RGI60-XXXX/')
# z1_fn_sample = 'surface_DEM_RGI60-XXXX.tif'
# # Ice thickness
# huss_dir_sample = ('/Users/davidrounce/Documents/Dave_Rounce/HiMAT/IceThickness_Farinotti/' + 
#                    'composite_thickness_RGI60-all_regions/RGI60-XXXX/')
# huss_fn_sample = 'RGI60-XXXX_thickness.tif'


    
# outdir_csv = debris_prms.outdir_emvel_fp 
# outdir_fig = debris_prms.outdir_emvel_fp  + '../figures/'

# if os.path.exists(glac_shp_proj_fp) == False:
#     os.makedirs(glac_shp_proj_fp)
# if os.path.exists(outdir_csv) == False:
#     os.makedirs(outdir_csv)
# if os.path.exists(outdir_fig) == False:
#     os.makedirs(outdir_fig)



/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/debris_global/../output/ts_tif/HMA_debris_tsinfo.nc


In [4]:
rgiid_list = []
rgiid_fn_list = []
for i in os.listdir(debris_prms.mb_binned_fp):
    if i.endswith('mb_bins.csv'):
        region = int(i.split('.')[0])
        if region in debris_prms.roi_rgidict[debris_prms.roi]:    
            if region < 10:
                rgiid_list.append(i[0:7])
            else:
                rgiid_list.append(i[0:8])
            rgiid_fn_list.append(i)
        
        
rgiid_list = sorted(rgiid_list)
rgiid_fn_list = sorted(rgiid_fn_list)

print(len(rgiid_list))

main_glac_rgi = debris_prms.selectglaciersrgitable(rgiid_list)
main_glac_rgi['CenLon_360'] = main_glac_rgi['CenLon']
main_glac_rgi.loc[main_glac_rgi['CenLon_360'] < 0, 'CenLon_360'] = (
    360 + main_glac_rgi.loc[main_glac_rgi['CenLon_360'] < 0, 'CenLon_360'])
main_glac_rgi['bin_fn'] = rgiid_fn_list

2958
1090 glaciers in region 13 are included in this model run: ['00604', '00611', '00643', '00713', '00757', '00761', '00763', '00777', '00788', '00809', '00830', '00834', '00838', '00880', '00884', '00885', '00891', '00905', '00906', '00940', '00949', '00951', '00954', '00956', '00964', '00965', '00967', '00972', '00982', '00995', '00997', '00999', '01019', '01022', '01023', '01027', '01038', '01044', '01045', '01050', '01098', '01099', '01113', '01124', '01129', '01136', '01144', '01145', '01148', '01150'] and more
1042 glaciers in region 14 are included in this model run: ['00005', '00018', '00032', '00036', '00043', '00057', '00072', '00104', '00145', '00163', '00222', '00287', '00353', '00363', '00471', '00543', '00548', '00555', '00595', '00700', '00722', '00742', '00764', '00767', '00796', '00805', '00850', '00891', '00899', '00952', '01001', '01022', '01070', '01075', '01165', '01191', '01206', '01226', '01228', '01244', '01285', '01361', '01379', '01391', '01400', '01409', '0

In [5]:
# Merge with debris cover stats
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_shp = dc_shp.sort_values(by=['RGIId'])
dc_shp.reset_index(inplace=True, drop=True)

dc_areaperc_dict = dict(zip(dc_shp.RGIId.values,dc_shp['DC_Area_%'].values))
main_glac_rgi['DC_Area_%'] = main_glac_rgi.RGIId.map(dc_areaperc_dict).fillna(0)
dc_area_dict = dict(zip(dc_shp.RGIId.values,dc_shp['DC_Area_v2'].values))
main_glac_rgi['DC_Area_v2'] = main_glac_rgi.RGIId.map(dc_area_dict).fillna(0)
main_glac_rgi

,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,...,TermType,Surging,RefDate,glacno,rgino_str,RGIId_float,CenLon_360,bin_fn,DC_Area_%,DC_Area_v2
GlacNo,,,,,,,,,,,,,,,,,,,,,
0,603,RGI60-13.00604,77.996700,35.568400,13,5,9.855,5401,6120,5778,...,0,9,20020802,604,13.00604,13.00604,77.996700,13.00604_mb_bins.csv,6.274,498099
1,610,RGI60-13.00611,94.297646,30.361333,13,9,2.035,4133,5756,5092,...,0,9,19990721,611,13.00611,13.00611,94.297646,13.00611_mb_bins.csv,16.629,339441
2,642,RGI60-13.00643,94.928018,30.607432,13,9,28.533,4345,6187,5212,...,0,9,19990923,643,13.00643,13.00643,94.928018,13.00643_mb_bins.csv,11.582,3118320
3,712,RGI60-13.00713,94.777072,30.796392,13,9,2.996,5085,5738,5428,...,0,9,19990923,713,13.00713,13.00713,94.777072,13.00713_mb_bins.csv,6.459,168314
4,756,RGI60-13.00757,94.632462,30.673603,13,9,2.887,4619,5750,5231,...,0,9,19990923,757,13.00757,13.00757,94.632462,13.00757_mb_bins.csv,8.324,197136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2953,13060,RGI60-15.13061,97.506000,28.969000,15,3,4.332,4608,5569,5076,...,0,9,20091014,13061,15.13061,15.13061,97.506000,15.13061_mb_bins.csv,23.622,940764
2954,13064,RGI60-15.13065,97.526000,28.985000,15,3,6.516,4141,5680,5133,...,0,9,20091014,13065,15.13065,15.13065,97.526000,15.13065_mb_bins.csv,6.464,385319
2955,13066,RGI60-15.13067,97.536000,29.002000,15,3,2.788,4445,5456,5113,...,0,9,20091014,13067,15.13067,15.13067,97.536000,15.13067_mb_bins.csv,8.651,171952


In [6]:
# Latitude and longitude index to run the model
#  Longitude must be 0 - 360 degrees
latlon_all = []
for i in os.listdir(debris_prms.ostrem_fp):
    if i.endswith(debris_prms.ostrem_fn_sample.split('XXXX')[1]):
        latlon_fn = i.split(debris_prms.ostrem_fn_sample.split('XXXX')[1])[0]
        # Extract latitude
        lat_str = latlon_fn.split('-')[0]
        if 'N' in lat_str:
            i_lat = int(lat_str.split('N')[0]) / 100
        elif 'S' in lat_str:
            i_lat = -1 * int(lat_str.split('S')[0]) / 100
        # Extract longitude
        lon_str = latlon_fn.split('-')[1]
        i_lon = int(lon_str.split('E')[0]) / 100
        latlon_all.append([i_lat, i_lon, i])
latlon_all = sorted(latlon_all)

lat_all = np.array([x[0] for x in latlon_all])
lon_all = np.array([x[1] for x in latlon_all])
ostrem_fn_all_raw = [x[2] for x in latlon_all]

main_glac_rgi['lat_nearest'] = np.nan
main_glac_rgi['lon_nearest'] = np.nan
main_glac_rgi['ostrem_fn'] = np.nan
for nglac, glac_idx in enumerate(main_glac_rgi.index.values):
    latlon_dist = (((main_glac_rgi.loc[glac_idx,'CenLat'] - lat_all)**2 + 
                    (main_glac_rgi.loc[glac_idx,'CenLon_360'] - lon_all)**2)**0.5)
    latlon_nearidx = np.where(latlon_dist == latlon_dist.min())[0][0]
    
    main_glac_rgi.loc[glac_idx,'lat_nearest'] = lat_all[latlon_nearidx]
    main_glac_rgi.loc[glac_idx,'lon_nearest'] = lon_all[latlon_nearidx]
    main_glac_rgi.loc[glac_idx,'ostrem_fn'] = ostrem_fn_all_raw[latlon_nearidx]
    
ostrem_fn_all = sorted(list(np.unique(main_glac_rgi['ostrem_fn'].values)))
lat_values = np.arange(main_glac_rgi.lat_nearest.min(), main_glac_rgi.lat_nearest.max() + 0.1, 0.25)
lon_values = np.arange(main_glac_rgi.lon_nearest.min(), main_glac_rgi.lon_nearest.max() + 0.1, 0.25)

main_glac_rgi

,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,...,glacno,rgino_str,RGIId_float,CenLon_360,bin_fn,DC_Area_%,DC_Area_v2,lat_nearest,lon_nearest,ostrem_fn
GlacNo,,,,,,,,,,,,,,,,,,,,,
0,603,RGI60-13.00604,77.996700,35.568400,13,5,9.855,5401,6120,5778,...,604,13.00604,13.00604,77.996700,13.00604_mb_bins.csv,6.274,498099,35.25,77.75,3525N-7775E-_debris_melt_curve.nc
1,610,RGI60-13.00611,94.297646,30.361333,13,9,2.035,4133,5756,5092,...,611,13.00611,13.00611,94.297646,13.00611_mb_bins.csv,16.629,339441,30.25,94.25,3025N-9425E-_debris_melt_curve.nc
2,642,RGI60-13.00643,94.928018,30.607432,13,9,28.533,4345,6187,5212,...,643,13.00643,13.00643,94.928018,13.00643_mb_bins.csv,11.582,3118320,30.50,95.00,3050N-9500E-_debris_melt_curve.nc
3,712,RGI60-13.00713,94.777072,30.796392,13,9,2.996,5085,5738,5428,...,713,13.00713,13.00713,94.777072,13.00713_mb_bins.csv,6.459,168314,30.75,94.75,3075N-9475E-_debris_melt_curve.nc
4,756,RGI60-13.00757,94.632462,30.673603,13,9,2.887,4619,5750,5231,...,757,13.00757,13.00757,94.632462,13.00757_mb_bins.csv,8.324,197136,30.75,94.75,3075N-9475E-_debris_melt_curve.nc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2953,13060,RGI60-15.13061,97.506000,28.969000,15,3,4.332,4608,5569,5076,...,13061,15.13061,15.13061,97.506000,15.13061_mb_bins.csv,23.622,940764,29.00,97.50,2900N-9750E-_debris_melt_curve.nc
2954,13064,RGI60-15.13065,97.526000,28.985000,15,3,6.516,4141,5680,5133,...,13065,15.13065,15.13065,97.526000,15.13065_mb_bins.csv,6.464,385319,29.00,97.50,2900N-9750E-_debris_melt_curve.nc
2955,13066,RGI60-15.13067,97.536000,29.002000,15,3,2.788,4445,5456,5113,...,13067,15.13067,15.13067,97.536000,15.13067_mb_bins.csv,8.651,171952,29.00,97.50,2900N-9750E-_debris_melt_curve.nc


In [7]:
# Process each group and derive elevation statistics for the debris cover
year_mean = np.zeros((len(lat_values), len(lon_values)))
year_std = np.zeros((len(lat_values), len(lon_values)))
year_med = np.zeros((len(lat_values), len(lon_values)))
year_mad = np.zeros((len(lat_values), len(lon_values)))
doy_mean = np.zeros((len(lat_values), len(lon_values)))
doy_std = np.zeros((len(lat_values), len(lon_values)))
doy_med = np.zeros((len(lat_values), len(lon_values)))
doy_mad = np.zeros((len(lat_values), len(lon_values)))
dayfrac_mean = np.zeros((len(lat_values), len(lon_values)))
dayfrac_std = np.zeros((len(lat_values), len(lon_values)))
dayfrac_med = np.zeros((len(lat_values), len(lon_values)))
dayfrac_mad = np.zeros((len(lat_values), len(lon_values)))

for nlatlon, ostrem_fn in enumerate(ostrem_fn_all):
# for nlatlon, ostrem_fn in enumerate([ostrem_fn_all[10]]): # Miage
# for nlatlon, ostrem_fn in enumerate([ostrem_fn_all[0]]):
    
    main_glac_rgi_subset = main_glac_rgi[main_glac_rgi['ostrem_fn'] == ostrem_fn]
    main_glac_rgi_subset.reset_index(inplace=True, drop=True)
    
    lat_idx = np.where(main_glac_rgi_subset.loc[0,'lat_nearest'] == lat_values)[0][0]
    lon_idx = np.where(main_glac_rgi_subset.loc[0,'lon_nearest'] == lon_values)[0][0]
    
    print(nlatlon, lat_values[lat_idx], lon_values[lon_idx], ostrem_fn)

    doy_list = []
    year_list = []
    dayfrac_list = []
    for nglac, glac_idx in enumerate(main_glac_rgi_subset.index.values):
#     for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[0]]):
        glac_str = main_glac_rgi_subset.loc[glac_idx,'rgino_str']
        rgiid = main_glac_rgi_subset.loc[glac_idx,'RGIId']
        region = glac_str.split('.')[0]

#         print(nglac, glac_idx, rgiid,'\n')
        
        # Process debris-covered glaciers
        if ((main_glac_rgi_subset.loc[glac_idx, 'DC_Area_%'] > debris_prms.dc_percarea_threshold) | 
            (main_glac_rgi_subset.loc[glac_idx, 'DC_Area_v2'] / 1e6 > debris_prms.dc_area_threshold)):

            # Create glacier feature from ice thickness raster
            thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
            thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
            gf = create_glacfeat(thick_dir, thick_fn)

            # Debris shape layer processing
            dc_shp_proj_fn = (debris_prms.glac_shp_proj_fp + glac_str + '_dc_crs' + 
                              str(gf.aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp')
            if os.path.exists(dc_shp_proj_fn) == False:
                dc_shp_init = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
                dc_shp_single = dc_shp_init[dc_shp_init['RGIId'] == rgiid]
                dc_shp_single = dc_shp_single.reset_index()
                dc_shp_proj = dc_shp_single.to_crs({'init': 'epsg:' + str(gf.aea_srs.GetAttrValue("AUTHORITY", 1))})
                dc_shp_proj.to_file(dc_shp_proj_fn)
            dc_shp_ds = ogr.Open(dc_shp_proj_fn, 0)
            dc_shp_lyr = dc_shp_ds.GetLayer()

            # Add layers
            gf.add_layers(dc_shp_lyr, gf_add_dhdt=False, gf_add_vel=False, gf_add_ts=True, gf_add_ts_info=True)

            # Isolate the debris-covered areas
#             dc_shp_lyr_mask = geolib.lyr2mask(dc_shp_lyr, gf.ds_dict['ice_thick'])
#             dc_mask = np.ma.mask_or(dc_shp_lyr_mask, gf.H.mask)
            gf.ts.mask = gf.dc_mask
            gf.ts_dayfrac.mask = gf.dc_mask
            gf.ts_year.mask = gf.dc_mask
            gf.ts_doy.mask = gf.dc_mask

            # ===== PLOTS =====
#             # DEM
#             var_full2plot = gf.z1.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' DEM'], 'inferno', 'elev (masl)', close_fig=False)
#             # Surface temperature
#             var_full2plot = gf.ts.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'ts (degC)', close_fig=False)
#             # Ts Day fraction
#             var_full2plot = gf.ts_dayfrac.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' Ts-dayfrac'], 'inferno', 'ts dayfrac', close_fig=False)
#             # Ts Year
#             var_full2plot = gf.ts_year.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' Ts-year'], 'inferno', 'ts year', close_fig=False)
#             # Ts Day of year
#             var_full2plot = gf.ts_doy.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' Ts-doy'], 'inferno', 'doy', close_fig=False)
            
        # ========================================================================================================
                    
            doy_list.extend(list(gf.ts_doy.compressed()))
            year_list.extend(list(gf.ts_year.compressed()))
            dayfrac_list.extend(list(gf.ts_dayfrac.compressed()))
                
    
    # Compute statistics
    year_mean_latlon = np.mean(year_list)
    year_std_latlon = np.std(year_list)
    year_med_latlon = malib.fast_median(year_list)
    year_mad_latlon = malib.mad(year_list)
    doy_mean_latlon = np.mean(doy_list)
    doy_std_latlon = np.std(doy_list)
    doy_med_latlon = malib.fast_median(doy_list)
    doy_mad_latlon = malib.mad(doy_list)
    dayfrac_mean_latlon = np.mean(dayfrac_list)
    dayfrac_std_latlon = np.std(dayfrac_list)
    dayfrac_med_latlon = malib.fast_median(dayfrac_list)
    dayfrac_mad_latlon = malib.mad(dayfrac_list)
    
    # Update array
    year_mean[lat_idx,lon_idx] = year_mean_latlon
    year_std[lat_idx,lon_idx] = year_std_latlon
    year_med[lat_idx,lon_idx] = year_med_latlon
    year_mad[lat_idx,lon_idx] = year_mad_latlon
    doy_mean[lat_idx,lon_idx] = doy_mean_latlon
    doy_std[lat_idx,lon_idx] = doy_std_latlon
    doy_med[lat_idx,lon_idx] = doy_med_latlon
    doy_mad[lat_idx,lon_idx] = doy_mad_latlon
    dayfrac_mean[lat_idx,lon_idx] = dayfrac_mean_latlon
    dayfrac_std[lat_idx,lon_idx] = dayfrac_std_latlon
    dayfrac_med[lat_idx,lon_idx] = dayfrac_med_latlon
    dayfrac_mad[lat_idx,lon_idx] = dayfrac_mad_latlon
    
    print('  year mean +/- std:', np.round(year_mean_latlon,1), np.round(year_std_latlon,1)) 
    print('  doy mean +/- std:', np.round(doy_mean_latlon,1), np.round(doy_std_latlon,1)) 
    print('    doy median +/- mad:', np.round(doy_med_latlon,1), np.round(doy_mad_latlon,1)) 
    print('  dayfrac mean +/- std:', np.round(dayfrac_mean_latlon,3), np.round(dayfrac_std_latlon,3))

0 27.5 88.0 2750N-8800E-_debris_melt_curve.nc
  year mean +/- std: 2013.0 0.0
  doy mean +/- std: 163.2 0.0
    doy median +/- mad: 163.2 0.0
  dayfrac mean +/- std: 4.633 0.0
1 27.5 88.25 2750N-8825E-_debris_melt_curve.nc
  year mean +/- std: 2013.0 0.0
  doy mean +/- std: 163.2 0.0
    doy median +/- mad: 163.2 0.0
  dayfrac mean +/- std: 4.633 0.0
2 27.75 86.5 2775N-8650E-_debris_melt_curve.nc
  year mean +/- std: 2015.0 0.0
  doy mean +/- std: 272.2 0.0
    doy median +/- mad: 272.2 0.0
  dayfrac mean +/- std: 4.703 0.0
3 27.75 86.75 2775N-8675E-_debris_melt_curve.nc
  year mean +/- std: 2015.0 0.0
  doy mean +/- std: 272.2 0.0
    doy median +/- mad: 272.2 0.0
  dayfrac mean +/- std: 4.703 0.0
4 27.75 87.0 2775N-8700E-_debris_melt_curve.nc
  year mean +/- std: 2015.0 0.0
  doy mean +/- std: 272.2 0.0
    doy median +/- mad: 272.2 0.0
  dayfrac mean +/- std: 4.703 0.0
5 27.75 87.25 2775N-8725E-_debris_melt_curve.nc
  year mean +/- std: 2015.0 0.0
  doy mean +/- std: 272.2 0.0
    d

  year mean +/- std: 0.0 0.0
  doy mean +/- std: 0.0 0.0
    doy median +/- mad: 0.0 0.0
  dayfrac mean +/- std: 0.0 0.0
47 28.25 90.5 2825N-9050E-_debris_melt_curve.nc
  year mean +/- std: 0.0 0.0
  doy mean +/- std: 0.0 0.0
    doy median +/- mad: 0.0 0.0
  dayfrac mean +/- std: 0.0 0.0
48 28.25 90.75 2825N-9075E-_debris_melt_curve.nc
  year mean +/- std: 835.6 992.6
  doy mean +/- std: 92.3 112.4
    doy median +/- mad: 0.0 0.0
  dayfrac mean +/- std: 1.818 2.16
49 28.25 91.25 2825N-9125E-_debris_melt_curve.nc
  year mean +/- std: 2015.0 0.0
  doy mean +/- std: 251.2 0.1
    doy median +/- mad: 251.2 0.0
  dayfrac mean +/- std: 4.385 0.0
50 28.25 91.5 2825N-9150E-_debris_melt_curve.nc
  year mean +/- std: 2015.0 0.1
  doy mean +/- std: 250.1 8.1
    doy median +/- mad: 251.2 0.0
  dayfrac mean +/- std: 4.385 0.0
51 28.25 92.75 2825N-9275E-_debris_melt_curve.nc
  year mean +/- std: 2014.4 1.4
  doy mean +/- std: 243.8 26.8
    doy median +/- mad: 251.1 31.1
  dayfrac mean +/- std: 4.

  year mean +/- std: 2013.7 0.5
  doy mean +/- std: 236.6 40.4
    doy median +/- mad: 259.2 0.0
  dayfrac mean +/- std: 4.09 0.014
93 29.5 101.75 2950N-10175E-_debris_melt_curve.nc
  year mean +/- std: 2015.4 0.8
  doy mean +/- std: 205.5 31.2
    doy median +/- mad: 227.5 0.0
  dayfrac mean +/- std: 3.728 0.048
94 29.5 82.5 2950N-8250E-_debris_melt_curve.nc
  year mean +/- std: 2015.8 0.6
  doy mean +/- std: 237.2 27.4
    doy median +/- mad: 247.5 0.0
  dayfrac mean +/- std: 5.008 0.003
95 29.5 82.75 2950N-8275E-_debris_melt_curve.nc
  year mean +/- std: 2015.5 0.8
  doy mean +/- std: 226.9 35.0
    doy median +/- mad: 247.5 0.0
  dayfrac mean +/- std: 5.007 0.003
96 29.5 95.0 2950N-9500E-_debris_melt_curve.nc
  year mean +/- std: 2014.7 0.6
  doy mean +/- std: 211.9 9.1
    doy median +/- mad: 206.7 3.7
  dayfrac mean +/- std: 4.176 0.014
97 29.5 95.25 2950N-9525E-_debris_melt_curve.nc
  year mean +/- std: 2014.8 0.4
  doy mean +/- std: 206.0 2.2
    doy median +/- mad: 205.2 0.1
 

  year mean +/- std: 2015.0 0.6
  doy mean +/- std: 224.8 32.7
    doy median +/- mad: 214.1 43.5
  dayfrac mean +/- std: 5.126 0.047
139 30.25 80.75 3025N-8075E-_debris_melt_curve.nc
  year mean +/- std: 2015.0 0.6
  doy mean +/- std: 242.9 24.8
    doy median +/- mad: 252.8 17.4
  dayfrac mean +/- std: 5.121 0.042
140 30.25 81.5 3025N-8150E-_debris_melt_curve.nc
  year mean +/- std: 2014.8 0.4
  doy mean +/- std: 239.3 24.3
    doy median +/- mad: 252.1 18.4
  dayfrac mean +/- std: 5.099 0.003
141 30.25 81.75 3025N-8175E-_debris_melt_curve.nc
  year mean +/- std: 2015.0 0.1
  doy mean +/- std: 220.4 21.8
    doy median +/- mad: 204.3 4.4
  dayfrac mean +/- std: 5.096 0.002
142 30.25 82.0 3025N-8200E-_debris_melt_curve.nc
  year mean +/- std: 2015.3 1.0
  doy mean +/- std: 229.3 30.6
    doy median +/- mad: 247.5 5.6
  dayfrac mean +/- std: 5.032 0.04
143 30.25 82.25 3025N-8225E-_debris_melt_curve.nc
  year mean +/- std: 2013.6 1.0
  doy mean +/- std: 179.5 33.8
    doy median +/- mad

  year mean +/- std: 2014.9 1.0
  doy mean +/- std: 209.5 26.7
    doy median +/- mad: 209.9 31.6
  dayfrac mean +/- std: 5.295 0.021
185 31.0 79.25 3100N-7925E-_debris_melt_curve.nc
  year mean +/- std: 2014.9 0.9
  doy mean +/- std: 217.4 26.3
    doy median +/- mad: 224.2 28.3
  dayfrac mean +/- std: 5.26 0.046
186 31.0 79.5 3100N-7950E-_debris_melt_curve.nc
  year mean +/- std: 2014.8 1.0
  doy mean +/- std: 234.1 20.2
    doy median +/- mad: 243.2 5.6
  dayfrac mean +/- std: 5.228 0.038
187 31.0 79.75 3100N-7975E-_debris_melt_curve.nc
  year mean +/- std: 2015.0 1.1
  doy mean +/- std: 248.6 12.8
    doy median +/- mad: 248.0 16.2
  dayfrac mean +/- std: 5.21 0.01
188 31.0 93.75 3100N-9375E-_debris_melt_curve.nc
  year mean +/- std: 2016.0 0.0
  doy mean +/- std: 230.5 0.3
    doy median +/- mad: 230.5 0.0
  dayfrac mean +/- std: 4.278 0.002
189 31.25 78.25 3125N-7825E-_debris_melt_curve.nc
  year mean +/- std: 2015.5 0.8
  doy mean +/- std: 208.6 24.8
    doy median +/- mad: 189.

...

  year mean +/- std: 2014.0 0.0
  doy mean +/- std: 211.1 3.4
    doy median +/- mad: 210.2 0.1
  dayfrac mean +/- std: 4.965 0.0
415 37.0 71.25 3700N-7125E-_debris_melt_curve.nc
  year mean +/- std: 2015.6 0.9
  doy mean +/- std: 209.7 5.7
    doy median +/- mad: 207.7 0.0
  dayfrac mean +/- std: 5.799 0.03
416 37.0 71.75 3700N-7175E-_debris_melt_curve.nc
  year mean +/- std: 2014.9 1.1
  doy mean +/- std: 212.2 9.9
    doy median +/- mad: 207.8 5.6
  dayfrac mean +/- std: 5.808 0.037
417 37.0 72.0 3700N-7200E-_debris_melt_curve.nc
  year mean +/- std: 2014.8 1.0
  doy mean +/- std: 212.5 7.6
    doy median +/- mad: 211.4 9.1
  dayfrac mean +/- std: 5.793 0.013
418 37.0 72.25 3700N-7225E-_debris_melt_curve.nc
  year mean +/- std: 2014.3 0.7
  doy mean +/- std: 212.3 8.2
    doy median +/- mad: 212.7 8.4
  dayfrac mean +/- std: 5.79 0.004
419 37.0 72.75 3700N-7275E-_debris_melt_curve.nc
  year mean +/- std: 2013.8 0.4
  doy mean +/- std: 203.4 4.8
    doy median +/- mad: 202.2 1.2
  da

  year mean +/- std: 2013.9 0.9
  doy mean +/- std: 216.8 11.1
    doy median +/- mad: 213.5 12.3
  dayfrac mean +/- std: 5.902 0.018
461 38.5 71.5 3850N-7150E-_debris_melt_curve.nc
  year mean +/- std: 2014.1 1.2
  doy mean +/- std: 214.3 21.3
    doy median +/- mad: 206.3 7.6
  dayfrac mean +/- std: 5.903 0.016
462 38.5 71.75 3850N-7175E-_debris_melt_curve.nc
  year mean +/- std: 2014.5 1.0
  doy mean +/- std: 209.3 12.1
    doy median +/- mad: 206.8 14.2
  dayfrac mean +/- std: 5.858 0.051
463 38.5 72.0 3850N-7200E-_debris_melt_curve.nc
  year mean +/- std: 2014.6 0.9
  doy mean +/- std: 210.5 10.0
    doy median +/- mad: 207.8 11.7
  dayfrac mean +/- std: 5.81 0.041
464 38.5 72.25 3850N-7225E-_debris_melt_curve.nc
  year mean +/- std: 2014.9 1.0
  doy mean +/- std: 207.2 8.3
    doy median +/- mad: 207.2 7.2
  dayfrac mean +/- std: 5.807 0.037
465 38.5 72.5 3850N-7250E-_debris_melt_curve.nc
  year mean +/- std: 2015.1 1.0
  doy mean +/- std: 206.3 7.6
    doy median +/- mad: 207.1 

  year mean +/- std: 2014.8 1.0
  doy mean +/- std: 198.3 8.5
    doy median +/- mad: 200.8 5.3
  dayfrac mean +/- std: 5.783 0.008
507 39.25 73.25 3925N-7325E-_debris_melt_curve.nc
  year mean +/- std: 2014.1 0.3
  doy mean +/- std: 206.2 7.7
    doy median +/- mad: 202.2 0.0
  dayfrac mean +/- std: 5.781 0.001
508 39.25 73.5 3925N-7350E-_debris_melt_curve.nc
  year mean +/- std: 2013.4 0.9
  doy mean +/- std: 206.1 5.5
    doy median +/- mad: 208.2 0.0
  dayfrac mean +/- std: 5.724 0.026
509 39.25 74.25 3925N-7425E-_debris_melt_curve.nc
  year mean +/- std: 2013.4 0.6
  doy mean +/- std: 204.4 10.5
    doy median +/- mad: 208.2 0.8
  dayfrac mean +/- std: 5.701 0.017
510 39.25 74.5 3925N-7450E-_debris_melt_curve.nc
  year mean +/- std: 2013.9 1.1
  doy mean +/- std: 185.6 19.7
    doy median +/- mad: 195.2 19.3
  dayfrac mean +/- std: 5.695 0.017
511 39.25 74.75 3925N-7475E-_debris_melt_curve.nc
  year mean +/- std: 2013.2 0.5
  doy mean +/- std: 201.8 16.5
    doy median +/- mad: 20

  year mean +/- std: 2013.7 1.0
  doy mean +/- std: 212.0 10.5
    doy median +/- mad: 217.2 1.8
  dayfrac mean +/- std: 5.585 0.017
553 42.0 77.5 4200N-7750E-_debris_melt_curve.nc
  year mean +/- std: 2013.8 0.7
  doy mean +/- std: 196.6 14.3
    doy median +/- mad: 194.8 20.4
  dayfrac mean +/- std: 5.481 0.03
554 42.0 77.75 4200N-7775E-_debris_melt_curve.nc
  year mean +/- std: 2013.8 0.7
  doy mean +/- std: 200.8 17.4
    doy median +/- mad: 203.3 21.1
  dayfrac mean +/- std: 5.482 0.034
555 42.0 78.0 4200N-7800E-_debris_melt_curve.nc
  year mean +/- std: 2013.7 0.8
  doy mean +/- std: 201.9 13.3
    doy median +/- mad: 207.9 9.9
  dayfrac mean +/- std: 5.488 0.031
556 42.0 78.25 4200N-7825E-_debris_melt_curve.nc
  year mean +/- std: 2013.8 0.7
  doy mean +/- std: 199.6 20.5
    doy median +/- mad: 197.4 24.0
  dayfrac mean +/- std: 5.471 0.019
557 42.0 78.5 4200N-7850E-_debris_melt_curve.nc
  year mean +/- std: 2014.0 0.2
  doy mean +/- std: 182.2 5.3
    doy median +/- mad: 181.2

  year mean +/- std: 2015.0 0.2
  doy mean +/- std: 207.4 1.1
    doy median +/- mad: 207.2 0.0
  dayfrac mean +/- std: 5.551 0.004
599 43.5 85.0 4350N-8500E-_debris_melt_curve.nc
  year mean +/- std: 2014.3 0.5
  doy mean +/- std: 200.6 3.8
    doy median +/- mad: 201.2 0.0
  dayfrac mean +/- std: 5.02 0.041
600 43.75 84.5 4375N-8450E-_debris_melt_curve.nc
  year mean +/- std: 2014.9 0.3
  doy mean +/- std: 195.7 1.5
    doy median +/- mad: 195.2 0.0
  dayfrac mean +/- std: 5.126 0.025
601 43.75 84.75 4375N-8475E-_debris_melt_curve.nc
  year mean +/- std: 2015.0 0.2
  doy mean +/- std: 195.4 1.4
    doy median +/- mad: 195.2 0.0
  dayfrac mean +/- std: 5.13 0.017
602 44.0 83.5 4400N-8350E-_debris_melt_curve.nc
  year mean +/- std: 2015.0 0.1
  doy mean +/- std: 195.7 2.4
    doy median +/- mad: 195.2 0.0
  dayfrac mean +/- std: 5.133 0.003
603 44.0 83.75 4400N-8375E-_debris_melt_curve.nc
  year mean +/- std: 2015.0 0.0
  doy mean +/- std: 195.3 1.4
    doy median +/- mad: 195.2 0.0
  

In [8]:
# Export to dataset
ds_ts_stats = xr.Dataset({'year_mean': (['latitude', 'longitude'], year_mean),
                          'year_std': (['latitude', 'longitude'], year_std),
                          'year_med': (['latitude', 'longitude'], year_med),
                          'year_mad': (['latitude', 'longitude'], year_mad),
                          'doy_mean': (['latitude', 'longitude'], doy_mean),
                          'doy_std': (['latitude', 'longitude'], doy_std),
                          'doy_med': (['latitude', 'longitude'], doy_med),
                          'doy_mad': (['latitude', 'longitude'], doy_mad),
                          'dayfrac_mean': (['latitude', 'longitude'], dayfrac_mean),
                          'dayfrac_std': (['latitude', 'longitude'], dayfrac_std),
                          'dayfrac_med': (['latitude', 'longitude'], dayfrac_med),
                          'dayfrac_mad': (['latitude', 'longitude'], dayfrac_mad),},
                          coords={'latitude': lat_values,
                                  'longitude': lon_values})

attrs_dict={
     'year_mean':{'units':'-',
         'long_name':'mean year',
         'comment': 'mean year when mosaicked surface temperature satellite image was acquired'},
     'year_std':{'units':'-',
         'long_name':'year standard deviation',
         'comment': 'standard deviation of year when mosaicked surface temperature satellite image was acquired'},
     'year_med':{'units':'-',
         'long_name':'median year',
         'comment': 'median year when mosaicked surface temperature satellite image was acquired'},
     'year_mad':{'units':'-',
         'long_name':'median absolute deviation year',
         'comment': 'median absolute deviation of year of when mosaicked surface temperature satellite image was acquired'},
     'doy_mean':{'units':'days since January 1',
         'long_name':'mean day of year',
         'comment': 'mean day of year when mosaicked surface temperature satellite image was acquired'},
     'doy_std':{'units':'days since January 1',
         'long_name':'day of year standard deviation',
         'comment': 'standard deviation of day of year when mosaicked surface temperature satellite image was acquired'},
     'doy_med':{'units':'days since January 1',
         'long_name':'median day of year',
         'comment': 'median day of year when mosaicked surface temperature satellite image was acquired'},
     'doy_mad':{'units':'days since January 1',
         'long_name':'median absolute deviation day of year',
         'comment': 'day of year of year of when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_mean':{'units':'-',
         'long_name':'mean hour',
         'comment': 'mean hour of when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_std':{'units':'-',
         'long_name':'year standard deviation',
         'comment': 'standard deviation of hour when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_med':{'units':'-',
         'long_name':'median hour',
         'comment': 'median hour when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_mad':{'units':'-',
         'long_name':'median absolute deviation hour',
         'comment': 'median absolute deviation of hour of when mosaicked surface temperature satellite image was acquired'},}

for vn in ['year_mean', 'year_std', 'year_med', 'year_mad',
           'doy_mean', 'doy_std', 'doy_med', 'doy_mad',
           'dayfrac_mean', 'dayfrac_std', 'dayfrac_med', 'dayfrac_mad',]:
    ds_ts_stats[vn].attrs = attrs_dict[vn]
    
ds_ts_stats.to_netcdf(ts_info_fullfn)
                
print(ds_ts_stats)

<xarray.Dataset>
Dimensions:       (latitude: 70, longitude: 135)
Coordinates:
  * latitude      (latitude) float64 27.5 27.75 28.0 28.25 ... 44.25 44.5 44.75
  * longitude     (longitude) float64 68.5 68.75 69.0 ... 101.5 101.8 102.0
Data variables:
    year_mean     (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    year_std      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    year_med      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    year_mad      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_mean      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_std       (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_med       (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_mad       (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    dayfrac_mean  (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    dayfrac_std   (latitude, 

In [11]:
# lat_idx = 4
# lon_idx = 2
# # lat_idx = 37
# # lon_idx = 46
# print(ds_ts_stats['year_mean'][lat_idx,lon_idx].values, ds_ts_stats['year_std'][lat_idx,lon_idx].values, 
#       ds_ts_stats['doy_mean'][lat_idx,lon_idx].values, ds_ts_stats['doy_std'][lat_idx,lon_idx].values)

In [12]:
ts_info_fullfn

'/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/debris_global/../output/ts_tif/18_debris_tsinfo.nc'